# Solving for static equilibrium
This notebook will help you assess in simulation which of the sphere configurations in the problem represent configurations at equilibrium and which. **You do not need to turn in this notebook, and there is no autograded component.** It is just to help you build intuition, show you how to use Drake for problems like this, and check your answers!

## Imports and function definitions

In [ ]:
# python libraries
import numpy as np

import pydrake
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, FixedOffsetFrame, MeshcatVisualizerCpp,
    RigidTransform, RotationMatrix, Simulator, Solve, Sphere, StaticEquilibriumProblem
)

from manipulation.meshcat_cpp_utils import StartMeshcat
from manipulation.scenarios import AddShape
from manipulation import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

## Initialization

In [ ]:
mu = 0.5
r = 0.3
m = 1

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
plant.set_name("plant")

world_offset_frame = pydrake.multibody.tree.FixedOffsetFrame(
                        "world_joint_frame",
                        plant.world_frame(),
                        RigidTransform(
                            RotationMatrix.MakeXRotation(np.pi/2),
                            [0, 0, 0]))
plant.AddFrame(world_offset_frame)

# Create the sphere bodies
spheres = []
sphere_joints = []
for i in range(3):
    sphere_name = "sphere_{}".format(i)

    color = [0, 0, 0, 1]
    color[i] = 1
    spheres.append(AddShape(plant, pydrake.geometry.Sphere(r), name=sphere_name, mass=m, mu=mu, color=color))

    # Set up planar joint
    sphere_joints.append(plant.AddJoint(pydrake.multibody.tree.PlanarJoint(
        "sphere_{}_joint".format(i),
        world_offset_frame,
        plant.GetFrameByName(sphere_name))))

ground = AddShape(plant, pydrake.geometry.Box(10,10,2.0), name="ground", mu=mu)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("ground"), RigidTransform(p=[0,0,-1.0]))

plant.Finalize()

visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)

# Using the plant
This is the main of the notebook for you to edit. (The other spot is where the system parameters are defined near the top of the script.) There are three sections:

1. **Initializing your guess for a static equilibrium position**: You can specify the $xyz$ position of each of the sphere. (To answer the question, you'll want to make it match one of the configurations from the problem, but feel free to experiment/try others.)
2. **Computing the static equilibrium position**: The `StaticEquilibriumProblem` class allows us to automatically set up the optimization problem for static equilibrium for a given plant. We use this class to compute an actual equilibrium position.
3. **Simulating the plant.** Given a configuration for the system, simulate how it evolves over time.

## Initializing your guess for a static equilibrium position
Specify the x and z of the center of mass of each of the spheres. (The spheres are fixed in the $xz$ plane, so that's all you have to specify.)

In [ ]:
#########
# REPLACE WITH YOUR CODE
guesses = [
    [0, r], # Red sphere xz
    [2*r, r], # Green sphere xz
    [4*r, r] # Blue sphere xz
]
#########

### Visualizing your guess
Run the following cell to see your guess rendered in meshcat. **This does not check for static equilibrium or run any physics simulation,** but it will let you verify you've set your pose how you intended.

In [ ]:
for i, guess in enumerate(guesses):
    sphere_joints[i].set_translation(plant_context, guess)
diagram.Publish(context)

## Computing the static equilibrium position
This cell computes a static equilibrium postion. If it's close to your original guess, then you initialized the system at equilibrium. If not, your guess is not an equilibrium.

In [ ]:
# The StaticEquilibriumProblem needs an "autodiff" version of the diagram/multibody plant to 
# use gradient-based optimization.
autodiff_diagram = diagram.ToAutoDiffXd()
autodiff_context = autodiff_diagram.CreateDefaultContext()
autodiff_plant = autodiff_diagram.GetSubsystemByName("plant")
static_equilibrium_problem = StaticEquilibriumProblem(autodiff_plant, autodiff_plant.GetMyContextFromRoot(autodiff_context), set())

initial_guess = np.zeros(plant.num_positions())

for i, guess in enumerate(guesses):
    initial_guess[3*i] = guess[0] # x
    initial_guess[3*i+1] = guess[1] # z

static_equilibrium_problem.get_mutable_prog().SetInitialGuess(
    static_equilibrium_problem.q_vars(), initial_guess)

result = Solve(static_equilibrium_problem.prog())
result.is_success()
q_sol = result.GetSolution(static_equilibrium_problem.q_vars())

for i, guess in enumerate(guesses):
    print("Guess for position of {}:".format(i), guess, "\tEquilibrium position of sphere {}:".format(i), q_sol[3*i:3*i+2])

# The StaticEquilibriumProblem needs an "autodiff" version of the diagram/multibody plant to 
# use gradient-based optimization.
autodiff_diagram = diagram.ToAutoDiffXd()
autodiff_context = autodiff_diagram.CreateDefaultContext()
autodiff_plant = autodiff_diagram.GetSubsystemByName("plant")
static_equilibrium_problem = StaticEquilibriumProblem(autodiff_plant, autodiff_plant.GetMyContextFromRoot(autodiff_context), set())

initial_guess = np.zeros(plant.num_positions())

for i, guess in enumerate(guesses):
    initial_guess[3*i] = guess[0] # x
    initial_guess[3*i+1] = guess[1] # z

static_equilibrium_problem.get_mutable_prog().SetInitialGuess(
    static_equilibrium_problem.q_vars(), initial_guess)

result = Solve(static_equilibrium_problem.prog())
result.is_success()
q_sol = result.GetSolution(static_equilibrium_problem.q_vars())

for i, guess in enumerate(guesses):
    print("Guess for position of {}:".format(i), guess, "\tEquilibrium position of sphere {}:".format(i), q_sol[3*i:3*i+2])

for wrench in static_equilibrium_problem.GetContactWrenchSolution(result):
    print(f"Spatial force at world position {wrench.p_WCb_W} between {wrench.bodyA_index} and {wrench.bodyB_index}:")
    print(f"  translational: {wrench.F_Cb_W.translational()}")
    print(f"  rotational: {wrench.F_Cb_W.rotational()}")

### Visualizing the solution configuration
This doesn't yet run the dynamics for the system (so the objects won't move), but it *will* update their poses to match the results of `StaticEquilibriumProblem`.

In [ ]:
plant.SetPositions(plant_context, q_sol)
diagram.Publish(context)

## Simulating the solution

You may see simulations of the static equilibrium that result in the spheres moving.  Why is that?

Keep in mind that
- A static equilibrium might not be a *stable* equilibrium.  States close to the equilibrium might diverge.
- The optimization solver satisfies the equations only up to a numerical tolerance.

In [ ]:
simulator = Simulator(diagram)
plant.SetPositions(plant.GetMyContextFromRoot(simulator.get_mutable_context()), q_sol)
if running_as_notebook:
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(5.0);
else:
  simulator.AdvanceTo(0.1);

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=da179554-1a2d-4268-85aa-b1e5b071712b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>